# Introduction

texte explicatif du contenu du code et le but de ce projet

# Imports

## Mounting the drive

In [ ]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#drive path
drive_path = "drive/My Drive/"

## Importing the images

In [ ]:
#unzipping the images from the drive into the local environment
import zipfile
with zipfile.ZipFile(drive_path+"ProjetDL/images.zip","r") as zip_ref:
  zip_ref.extractall("lesimages/")

In [ ]:
#checking the loading
import os
root = 'lesimages/img_align_celeba'
img_list = os.listdir(root)
print(len(img_list))

## Other imports needed

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils import data
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from PIL import Image
import torch
import os
import random
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.nn import Linear, Conv2d, BatchNorm2d, MaxPool2d, Dropout2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
import torch.optim as optim
from google.colab.patches import cv2_imshow
import cv2

# Creating the sets

## The DataLoader

dire qu'on s'est basé sur le repo github

In [ ]:
class CelebA(data.Dataset):
    """Dataset class for the CelebA dataset."""

    def __init__(self, image_dir, attr_path,  split_path, selected_attrs, transform, mode):
        """Initialize and preprocess the CelebA dataset."""
        self.image_dir = image_dir
        self.attr_path = attr_path
        self.split_path = split_path
        self.selected_attrs = selected_attrs
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.train_dataset_with_added_bald = []
        self.test_dataset = []
        self.valid_dataset = []
        self.all_dataset = []
        self.all_dataset_with_added_bald = []
        self.attr2idx = {}
        self.idx2attr = {}
        self.preprocess()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        elif mode == 'valid':
            self.num_images = len(self.valid_dataset)
        else:
            self.num_images = len(self.test_dataset)

    def preprocess(self):
        """Preprocess the CelebA attribute file."""
        dict_split = dict()
        lines = [line.rstrip() for line in open(self.split_path, 'r')]
        for line in lines:
          split = line.split()
          dict_split[split[0]]=int(split[1])

        lines = [line.rstrip() for line in open(self.attr_path, 'r')]
        all_attr_names = lines[1].split()
        for i, attr_name in enumerate(all_attr_names):
            self.attr2idx[attr_name] = i
            self.idx2attr[i] = attr_name

        lines = lines[2:]
        random.seed(1234)
        random.shuffle(lines)
        for i, line in enumerate(lines):
            split = line.split()
            filename = split[0]
            values = split[1:]

            label = []
            all_label = []

            for attr_name in self.attr2idx.keys():
                idx = self.attr2idx[attr_name]
                all_label.append(values[idx] == '1')
            for attr_name in self.selected_attrs:
                idx = self.attr2idx[attr_name]
                label.append(values[idx] == '1')

            if (self.mode == 'train'):
              if (dict_split[filename] == 0):

##### --- Uncomment the following lines in order to introduce data into de sets (repetition of images with a chosen attribute) --- #####
                # if all_label[4] == True:
                #   for i in range(1,10):
                #     im = Image.open("lesimages/img_align_celeba/"+filename) 
                #     im.save("lesimages/img_align_celeba/new_image_"+filename.split(".")[0]+str(i)+".png")
                #     self.train_dataset.append(["new_image_"+filename.split(".")[0]+str(i)+".png",label])
                #     self.all_dataset.append(["new_image_"+filename.split(".")[0]+str(i)+".png", all_label])

                self.train_dataset.append([filename, label])
                self.all_dataset.append([filename, all_label])
            elif (self.mode == 'valid'):
              if (dict_split[filename] == 1):
                self.valid_dataset.append((filename, label))
                self.all_dataset.append([filename, all_label])
            else:
              if (dict_split[filename] == 2):
                self.test_dataset.append((filename, label))
                self.all_dataset.append([filename, all_label])

        print('Finished preprocessing the CelebA dataset...')

    def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.valid_dataset if self.mode == 'valid' else self.test_dataset
        filename, label = dataset[index]
        image = Image.open(os.path.join(self.image_dir, filename))
        return self.transform(image), torch.FloatTensor(label)

    def __len__(self):
        """Return the number of images."""
        return self.num_images


def get_loader(image_dir, attr_path, selected_attrs,  split_path, crop_size=178, image_size=128, 
               batch_size=16, dataset='CelebA', mode='train', num_workers=1):
    """Build and return a data loader."""
    transform = []
    if mode == 'train':
        transform.append(T.RandomHorizontalFlip())
    transform.append(T.CenterCrop(crop_size))
    transform.append(T.Resize(image_size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    if dataset == 'CelebA':
        dataset = CelebA(image_dir, attr_path, split_path, selected_attrs, transform, mode)
    elif dataset == 'RaFD':
        dataset = ImageFolder(image_dir, transform)

    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
    return data_loader, dataset

## Creating the sets

### Common variables

In [ ]:
#image directiory :
celeba_image_dir = "lesimages/img_align_celeba/"
#split
split_path = drive_path+"ProjetDL/list_eval_partition.txt"
#attributes
attr_path = drive_path+"ProjetDL/list_attr_celeba.txt"
#selected attribute :
#selected_attrs = ['Attractive']
selected_attrs = ["Attractive"]
#crop size
celeba_crop_size = 178
#image size
image_size = 128
#batch_size
batch_size = 24
#num workers
num_workers = 1

### Train set

In [ ]:
mode = 'train'
trainloader, trainset = get_loader(celeba_image_dir, attr_path, selected_attrs, split_path,
                                   celeba_crop_size, image_size, batch_size,
                                   'CelebA', mode, num_workers)

trainset_allattr = trainset.all_dataset

### Validation set

In [ ]:
mode = 'valid'

validloader, validset = get_loader(celeba_image_dir, attr_path, selected_attrs, split_path,
                                   celeba_crop_size, image_size, batch_size,
                                   'CelebA', mode, num_workers)

### Test set

In [ ]:
mode = 'test'

testloader, testset = get_loader(celeba_image_dir, attr_path, selected_attrs, split_path,
                                   celeba_crop_size, image_size, batch_size,
                                   'CelebA', mode, num_workers)

testset_allattr = testset.all_dataset

## Printing some images


In [ ]:
train_data_iter = iter(trainloader)
test_data_iter = iter(testloader)

classes = ['Unattractive', 'Attractive']
def imshow(img):
    """ show an image """
    img = img / 2 + 0.5 # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
images, labels = train_data_iter.next()

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print('  '.join('%5s' % classes[int(labels[j].item())] for j in range(batch_size)))

# Creating the model

## Model variables

In [ ]:
x, y = next(iter(trainloader))
print("Batch dimension [B x C x H x W]:", x.shape)
channels_from_iter = x.shape[1]
height_from_iter = x.shape[2]
width_from_iter = x.shape[3]

#hyperparameters for the model
num_classes = len(selected_attrs) #number of classes depending on what we keep
#print(trainset.data.shape)
channels = channels_from_iter
#print(channels)
height = height_from_iter
#print(height)
width = width_from_iter
#print(width)

#first convolution
num_filters_conv1 = 96
kernel_size_conv1 = 3 # [height, width]
stride_conv1 = 1 # [stride_height, stride_width]
padding_conv1 = 1

#second convolution
num_filters_conv2 = 128
kernel_size_conv2 = 4 # [height, width]
stride_conv2 = 2 # [stride_height, stride_width]
padding_conv2 = 2

#hidden layer
num_l1 = 96

## Defining the model

In [ ]:
def compute_conv_dim(dim_size):
    return int((dim_size - kernel_size_conv2 + 2 * padding_conv2) / stride_conv2 + 1)

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.num_classes = num_classes
        
        #first convolution
        self.conv_1 = Conv2d(in_channels=channels,
                             out_channels=num_filters_conv1,
                             kernel_size=kernel_size_conv1,
                             stride=stride_conv1,
                             padding=padding_conv1)
        
        self.batch_norm1 = BatchNorm2d(num_filters_conv1)
        

        self.conv_2 = Conv2d(in_channels=num_filters_conv1,
                             out_channels=num_filters_conv2,
                             kernel_size=kernel_size_conv2,
                             stride=stride_conv2,
                             padding=padding_conv2)
        
        self.batch_norm2 = BatchNorm2d(num_filters_conv2)
        
        self.conv_out_height = compute_conv_dim(height)
        self.conv_out_width = compute_conv_dim(width)


        self.l1_in_features = num_filters_conv2 * self.conv_out_height * self.conv_out_width
 

        #first layer
        self.l_1 = Linear(in_features=self.l1_in_features, #dense hidden layer
                          out_features=num_l1,
                          bias=True)
        self.dropout = Dropout2d(p=0.3)
        #output layer
        self.l_out = Linear(in_features=num_l1, 
                            out_features=num_classes,
                            bias=False)

    def forward(self, x):
      x = relu(self.conv_1(x))
      x = self.batch_norm1(x)
      x = relu(self.conv_2(x))
      x = self.batch_norm2(x)
      x = x.view(-1, self.l1_in_features)
      x = relu(self.l_1(x))
      x = self.dropout(x)
      return torch.sigmoid(self.l_out(x))
    

net = Net(num_classes)
net.cuda()
print(net)

# Training the model

## Defining a loss function

In [ ]:
criterion = nn.BCELoss()  # Your code here!
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

## Training the model

In [ ]:
num_epoch = 4  # Your code here!

train_iter = []
train_loss = []
train_accs = []
valid_iter = []
valid_loss = []
valid_accs = []

log_every = 1000
eval_every = 1000

from IPython.display import clear_output

for epoch in range(num_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    net.train()
    correct = 0
    total = 0

    correct_valid = 0
    total_valid = 0
    for i, data in enumerate(trainloader, 0):

##### --- Uncomment this part in order to do the validation as well --- #####
        # if (i % eval_every == 0):
        #   net.eval()
        #   val_losses,val_accs,val_length = 0,0,0
        #   with torch.no_grad():
        #     for j,data in enumerate(validloader,0):
        #       inputs_valid,labels_valid = data
        #       inputs_valid, labels_valid = inputs_valid.cuda(), labels_valid.cuda()

        #       output_valid = net(inputs_valid)
        #       predicted_valid = torch.round(output_valid.data)
        #       correct_valid += (predicted_valid == labels_valid).sum()
        #       total_valid += labels_valid.size(0)

        #       val_losses += criterion(output_valid,labels_valid).item()
        #       val_accs += 100 * correct_valid.true_divide(total_valid)
        #       val_length += 1 

        #     valid_iter.append(i+len(trainloader)*epoch)
        #     valid_loss.append(val_losses/val_length)
        #     valid_accs.append(val_accs/val_length)

        net.train()
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        #forward
        output = net(inputs)
        loss = criterion(output, labels)
        #backward
        loss.backward()

        #optimize
        optimizer.step()


        # print statistics
        running_loss += loss.item()

        predicted = torch.round(output.data)
        correct += (predicted == labels).sum()
        total += labels.size(0)

        if i %100 == 99:
          train_iter.append(i+len(trainloader)*epoch)
          train_loss.append(float(running_loss / 100))
          running_loss = 0.0
          accuracy = 100 * correct.true_divide(total)
          train_accs.append(float(accuracy))
          total = 0
          correct = 0

        if i % log_every == 0:
            fig = plt.figure(figsize=(12,4))
            plt.subplot(1, 2, 1)
            plt.plot(train_iter, train_loss, label='train_loss')

##### --- Uncomment the following line in order to plot the validation evolution --- #####
            # plt.plot(valid_iter, valid_loss, label='valid_loss')
            plt.xlabel("Number of batches")
            plt.ylabel("Mean Loss of the 100 previous batches")
            plt.title("Loss of the model over the batches")
            plt.legend()

            plt.subplot(1, 2, 2)
            plt.plot(train_iter, train_accs, label='train_accs')
          
##### --- Uncomment the following line in order to plot the validation evolution --- #####
            # plt.plot(valid_iter, valid_accs, label='valid_accs')
            plt.legend()
            plt.title("Accuracy of the model over the batches")
            plt.ylabel("Accuracy")
            plt.xlabel("Number of batches")
            plt.show()
            clear_output(wait=True)

# Loading the trained model

expliquer que comme ça prend pas mal de temps, on peut direct les télécharger

## The model trained without added data

In [ ]:
PATH = drive_path+"ProjetDL/testdata_without.pth"
net.load_state_dict(torch.load(PATH))

## The model trained with added data

In [ ]:
PATH = drive_path+"ProjetDL/testdata_with.pth"
net.load_state_dict(torch.load(PATH))

# Viewing the results

## Testing the model

un petit texte pour expliquer quel modèle charger + quoi commenter/decommenter dans le dataloader

### Creating the dataframe

In [ ]:
##### TEST DATAFRAME #####
test_df_without_data = pd.DataFrame(testset_allattr)
test_df_without_data.columns = ["image_link","attrs"]
test_df_without_data[[key for key in testset.attr2idx.keys()]] = test_df_without_data['attrs'].apply(pd.Series)
test_df_without_data.pop("attrs")
test_df_without_data["status"]=["Not_known_yet" for i in range(len(test_df_without_data.index))]
test_df_without_data["predicted_value"]=[str(0) for i in range(len(test_df_without_data.index))]

##### TRAIN DATAFRAME #####
train_df_without_data = pd.DataFrame(trainset_allattr)
train_df_without_data.columns = ["image_link","attrs"]
train_df_without_data[[key for key in trainset.attr2idx.keys()]] = train_df_without_data['attrs'].apply(pd.Series)
del train_df_without_data["attrs"]

### Accuracy of the model

In [ ]:
correct = 0
total = 0
idx = 0 
not_predicted_as_attractive_idx_without_data = []

net.eval()
for data in testloader:
    images, labels = data
    labels = labels.cuda()
    outputs = net(images.cuda())
    predicted_raw = outputs.data
    predicted = torch.round(outputs.data)
    predicted = predicted.cuda()
    total += labels.size(0)
    correct += (predicted == labels).sum()
    for predict,predict_raw, real in zip(predicted,predicted_raw,labels):
      test_df_without_data.at[idx,"predicted_value"] = str(predict_raw.item())
      if predict==real:
        if real==1:
          test_df_without_data.at[idx,"status"] = "True Positive"
        else:
          test_df_without_data.at[idx,"status"] = "True Negative"
      else:
        if real==1:
          test_df_without_data.at[idx,"status"] = "False Negative"
        else:
          test_df_without_data.at[idx,"status"] = "False Positive"        

      if predict==0:
        not_predicted_as_attractive_idx_without_data.append(idx)
      idx+=1

print(total)
print(correct)

print('Accuracy of the network on the {} test images: {:4.2f} %'.format(
    len(testset), 100 * correct.true_divide(total)))

In [ ]:
test_df_without_data['predicted_value']= test_df_without_data['predicted_value'].apply(lambda x : float(x))
attractive_predicted_df = test_df_without_data.drop(not_predicted_as_attractive_idx)
attractive_predicted_df.head()

## Best and worst classified pictures

**1. Least attractive**

In [ ]:
test_df_without_data.sort_values(by='predicted_value', ascending=False).head()

In [ ]:
#Least attractive pictures
path0 = r'lesimages/img_align_celeba/'+'195079.jpg'
img0 = cv2.imread(path0) 
cv2_imshow(img0)

path1 = r'lesimages/img_align_celeba/'+'183494.jpg'
img1 = cv2.imread(path1) 
cv2_imshow(img1)

path2 = r'lesimages/img_align_celeba/'+'200832.jpg'
img2 = cv2.imread(path2) 
cv2_imshow(img2)	

**2. Most attractive**

In [ ]:
test_df_without_data.sort_values(by='predicted_value').head()

In [ ]:
#Most attractive pictures
path0 = r'lesimages/img_align_celeba/'+'190946.jpg'
img0 = cv2.imread(path0) 
cv2_imshow(img0)

path1 = r'lesimages/img_align_celeba/'+'192318.jpg'
img1 = cv2.imread(path1) 
cv2_imshow(img1)

path2 = r'lesimages/img_align_celeba/'+'202063.jpg'
img2 = cv2.imread(path2) 
cv2_imshow(img2)	

## Attributes amongst populations

ici les bar plot des distributions des populations, des false positive etc

# Fairness evaluation

### Attribute "Young"

In [ ]:
test_df = test_df_without_data

print("----------STATISCTICS WITHOUT DATA ----------")

print("GROUP FAIRNESS REGARDING YOUNG")
#P(M(X)=1 | A=a) = P(M(X)=1 | B=b)
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Young"]==True)])/len(test_df[test_df["Young"]==True]))
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Young"]==False)])/len(test_df[test_df["Young"]==False]))
print("\n")
print("EQUALIZED ODDS")
print("\n")
print("True Positive ")
#P(M(X)=1 | A=a, y=1) = P(M(X)=1 | A=b, y=1)
print("--------------")
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Young"]==True)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Young"]==True)]))
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Young"]==False)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Young"]==False)]))
print("\n")
print("False Positive ")
#P(M(X)=1 | A=a, y=0) = P(M(X)=1 | A=b, y=0)
print("----------------")
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Young"]==True)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Young"]==True)]))
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Young"]==False)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Young"]==False)]))


### Attribute "Bald"

In [ ]:
test_df = test_df_with_data

print("----------STATISCTICS WITH DATA ----------")

print("GROUP FAIRNESS REGARDING Bald")
#P(M(X)=1 | A=a) = P(M(X)=1 | B=b)
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Bald"]==True)])/len(test_df[test_df["Bald"]==True]))
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Bald"]==False)])/len(test_df[test_df["Bald"]==False]))
print("\n")
print("EQUALIZED ODDS")
print("\n")
print("True Positive ")
#P(M(X)=1 | A=a, y=1) = P(M(X)=1 | A=b, y=1)
print("--------------")
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Bald"]==True)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Bald"]==True)]))
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Bald"]==False)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Bald"]==False)]))
print("\n")
print("False Positive ")
#P(M(X)=1 | A=a, y=0) = P(M(X)=1 | A=b, y=0)
print("----------------")
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Bald"]==True)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Bald"]==True)]))
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Bald"]==False)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Bald"]==False)]))

# First fix : Adjusting the treshold

## Calculating the rates

In [ ]:
 def get_rates(treshold, attr):
  #create the dataframe
  test_df = pd.DataFrame(testset_allattr)
  test_df.columns = ["image_link","attrs"]
  test_df[[key for key in testset.attr2idx.keys()]] = test_df['attrs'].apply(pd.Series)
  test_df.pop("attrs")
  test_df["status"]=["Not_known_yet" for i in range(len(test_df.index))]
  test_df.head()

  #do the test
  correct = 0
  total = 0
  idx = 0 
  not_predicted_as_attractive_idx = []

  net.eval()
  for data in testloader:
      images, labels = data
      labels = labels.cuda()
      outputs = net(images.cuda())
      #round at desired treshold round(x - treshold + 0.5)
      predicted = torch.round(outputs.data - treshold + 0.5) #treshold changes here
      predicted = predicted.cuda()
      total += labels.size(0)
      correct += (predicted == labels).sum()

      for predict,real in zip(predicted,labels):

        if predict==real:
          if real==1:
            test_df.at[idx,"status"] = "True Positive"
          else:
            test_df.at[idx,"status"] = "True Negative"
        else:
          if real==1:
            test_df.at[idx,"status"] = "False Negative"
          else:
            test_df.at[idx,"status"] = "False Positive"        

        if predict==0:
          not_predicted_as_attractive_idx.append(idx)
        idx+=1

  print("treshold : " + str(treshold)+'\n')
  print(total)
  print(correct)
  #ici nous on a pas testset.data
  print('Accuracy of the network on the {} test images: {:4.2f} %'.format(
      len(testset), 100 * correct.true_divide(total)))

  #false positive rate with attribute
  fp_w = len(test_df[(test_df["status"]=="False Positive") & (test_df[attr]==True)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df[attr]==True)])
  #true positive rate with attribute
  tp_w = len(test_df[(test_df["status"]=="True Positive") & (test_df[attr]==True)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df[attr]==True)])
  #false positive rate without attribute
  fp_wo = len(test_df[(test_df["status"]=="False Positive") & (test_df[attr]==False)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df[attr]==False)])
  #true positive rate without attribute
  tp_wo = len(test_df[(test_df["status"]=="True Positive") & (test_df[attr]==False)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df[attr]==False)])

  return tp_w, fp_w, tp_wo, fp_wo

## Plotting the ROC curves

In [ ]:
category = 'Eyeglasses'
i = 0
x_w = []
y_w = []
x_wo = []
y_wo = []
interval = np.arange(0,1,0.05)
for t in interval:
  result = get_rates(t, category)
  y_w.append(result[0])
  x_w.append(result[1])
  y_wo.append(result[2])
  x_wo.append(result[3])

plt.plot(x_w,y_w, label = category, color = 'pink' )
plt.plot(x_wo,y_wo, label = 'wo'+category, color = 'blue')
plt.legend(title='ROC curves for the Eyeglasses attribute')
plt.show()

## Finding the tresholds

In [ ]:
#Printing the rates
print('TPR for eyeglasses')
print(x_w)
print('FPR for eyeglasses')
print(y_w)
print('TPR without eyeglasses')
print(x_wo)
print('FPR without eyeglasses')
print(y_wo)

We choose tresholds so that the rates are equal

In [ ]:
#Choosing a treshold of 0.4 for people with eyeglasses
print(get_rates(0.4, "Eyeglasses"))
#with eye glasses :
#true positive rate : 0.46601941747572817
#false positive rate : 0.05227655986509275

In [ ]:
#Choosing a treshold of 0.4 for people without eyeglasses
print(get_rates(0.8, "Eyeglasses"))
#without eye glasses :
#true positive rate : 0.46074527820316485
#false positive rate : 0.04178869114665465

## Recalculating the fairness metrics

In [ ]:
print("----------STATISCTICS----------")

print("GROUP FAIRNESS REGARDING Eyeglasses")
#P(M(X)=1 | A=a) = P(M(X)=1 | B=b)
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Eyeglasses"]==True)])/len(test_df[test_df["Eyeglasses"]==True]))
print(len(test_df[(test_df["status"].isin(["False Positive","True Positive"])) & (test_df["Eyeglasses"]==False)])/len(test_df[test_df["Eyeglasses"]==False]))
print("\n")
print("EQUALIZED ODDS")
print("\n")
print("True Positive ")
#P(M(X)=1 | A=a, y=1) = P(M(X)=1 | A=b, y=1)
print("--------------")
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Eyeglasses"]==True)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Eyeglasses"]==True)]))
print(len(test_df[(test_df["status"]=="True Positive") & (test_df["Eyeglasses"]==False)])/len(test_df[(test_df["status"].isin(["False Negative","True Positive"])) & (test_df["Eyeglasses"]==False)]))
print("\n")
print("False Positive ")
#P(M(X)=1 | A=a, y=0) = P(M(X)=1 | A=b, y=0)
print("----------------")
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Eyeglasses"]==True)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Eyeglasses"]==True)]))
print(len(test_df[(test_df["status"]=="False Positive") & (test_df["Eyeglasses"]==False)])/len(test_df[(test_df["status"].isin(["True Negative","False Positive"])) & (test_df["Eyeglasses"]==False)]))

# Second fix : Adding data

## With added data

un petit texte pour expliquer quel modèle charger + quoi commenter/decommenter dans le dataloader

In [ ]:
##### TEST DATAFRAME #####
test_df_with_data = pd.DataFrame(testset_allattr)
test_df_with_data.columns = ["image_link","attrs"]
test_df_with_data[[key for key in testset.attr2idx.keys()]] = test_df_with_data['attrs'].apply(pd.Series)
test_df_with_data.pop("attrs")
test_df_with_data["status"]=["Not_known_yet" for i in range(len(test_df_with_data.index))]
test_df_with_data["predicted_value"]=[str(0) for i in range(len(test_df_with_data.index))]

##### TRAIN DATAFRAME #####
train_df_with_data = pd.DataFrame(trainset_allattr)
train_df_with_data.columns = ["image_link","attrs"]
train_df_with_data[[key for key in trainset.attr2idx.keys()]] = train_df_with_data['attrs'].apply(pd.Series)
del train_df_with_data["attrs"]

### Testing the model

In [ ]:
correct = 0
total = 0
idx = 0 
not_predicted_as_attractive_idx_with_data = []

net.eval()
for data in testloader:
    images, labels = data
    labels = labels.cuda()
    outputs = net(images.cuda())
    predicted_raw = outputs.data
    predicted = torch.round(outputs.data)
    predicted = predicted.cuda()
    total += labels.size(0)
    correct += (predicted == labels).sum()
    for predict,predict_raw, real in zip(predicted,predicted_raw,labels):
      test_df_with_data.at[idx,"predicted_value"] = str(predict_raw.item())
      if predict==real:
        if real==1:
          test_df_with_data.at[idx,"status"] = "True Positive"
        else:
          test_df_with_data.at[idx,"status"] = "True Negative"
      else:
        if real==1:
          test_df_with_data.at[idx,"status"] = "False Negative"
        else:
          test_df_with_data.at[idx,"status"] = "False Positive"        

      if predict==0:
        not_predicted_as_attractive_idx_with_data.append(idx)
      idx+=1

print(total)
print(correct)

print('Accuracy of the network on the {} test images: {:4.2f} %'.format(
    len(testset), 100 * correct.true_divide(total)))

## Comparison between with and without added data

### Proportion of attributes


In [ ]:
# Set up a grid of plots
fig = plt.figure(figsize=(10,5)) 

# Plot accidents depending on type
x_attr = list(trainset.attr2idx.keys())
x_attr.remove("Attractive")

i=1
barWidth = 0.25

for attr in x_attr:
    y_percentage_without_data = len(train_df_without_data[train_df_without_data[attr]==True])/len(train_df_without_data)
    y_percentage_with_data = len(train_df_with_data[train_df_with_data[attr]==True])/len(train_df_with_data)

    plt.bar(i,y_percentage_without_data,color='#B22222',width=barWidth,label="w/o adding data" if i==1 else "")
    plt.bar(i+barWidth,y_percentage_with_data,color='#F08080',width=barWidth,label="w adding data" if i==1 else "")
    i+=1

barWidth = 0.25

plt.xticks([r + barWidth/2 for r in range(1,len(x_attr)+1)], [attr for attr in x_attr],rotation=90)
plt.legend(prop={'size': 12})
plt.title("Proportion of selected attributes inside different population")
plt.show()